# Focal cell coordinate finder

In [1]:
import calculate_radial_analysis as calculate
import dataio

import glob
import os
import re
from tqdm.auto import tqdm
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from natsort import natsorted
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## 1) Event dictionary loader

Specify cell type to filter out unwanted apoptoses, want end result of event_dict that takes the format of :`event_dict = {event_ID:event_time}` 

where `event_ID` takes the format of: `GV{experiment_ID}_Pos{position_ID}_{cell_ID}_{cell_type}` 

For example: 

```
event_dict = {'GV0794_Pos4_393_GFP': '766',
             'GV0794_Pos7_364_GFP': '550',
             'GV0794_Pos7_457_GFP': '805',
             'GV0794_Pos7_369_GFP': '833',
             'GV0794_Pos7_292_GFP': '884',
             'GV0795_Pos4_358_GFP': '628',
             'GV0795_Pos4_18_GFP': '855',
             'GV0795_Pos4_1356_GFP': '992',
             'GV0795_Pos4_359_GFP': '1133',
             'GV0796_Pos4_542_GFP': '581',
             'GV0796_Pos4_1334_GFP': '719',
             'GV0796_Pos4_1586_GFP': '785'}
 ```
 
Function to apoptosis event dictionary from preexisting data store:

`dataio.apoptosis_list_loader(path_to_apop_lists, cell_type)`

Where the `path_to_apop_lists` is the directory containing the apoptosis information and the `cell_type` is either `'Scr'` or `'WT'`.

In [2]:
path_to_apop_lists = '/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/apoptoses/final/all'
event_dict = dataio.apoptosis_list_loader(path_to_apop_lists, 'WT')
event_dict

{'GV0815_Pos23_185_GFP': '555',
 'GV0815_Pos23_634_GFP': '705',
 'GV0816_Pos14_7_GFP': '139',
 'GV0816_Pos14_537_GFP': '849',
 'GV0816_Pos14_496_GFP': '982',
 'GV0820_Pos1_130_GFP': '202',
 'GV0820_Pos1_412_GFP': '717',
 'GV0820_Pos1_1427_GFP': '840',
 'GV0820_Pos1_503_GFP': '867',
 'GV0820_Pos1_368_GFP': '911',
 'GV0820_Pos1_1818_GFP': '1007',
 'GV0813_Pos1_886_GFP': '598',
 'GV0817_Pos12_363_GFP': '699',
 'GV0817_Pos12_1193_GFP': '1090',
 'GV0818_Pos12_187_GFP': '549',
 'GV0818_Pos12_378_GFP': '604',
 'GV0818_Pos12_863_GFP': '632',
 'GV0818_Pos12_932_GFP': '728',
 'GV0818_Pos12_933_GFP': '728',
 'GV0818_Pos12_1099_GFP': '712',
 'GV0818_Pos12_957_GFP': '756',
 'GV0818_Pos12_1928_GFP': '933',
 'GV0818_Pos12_2092_GFP': '954',
 'GV0818_Pos12_1715_GFP': '930',
 'GV0818_Pos12_1714_GFP': '984',
 'GV0818_Pos12_1360_GFP': '1013',
 'GV0818_Pos12_2659_GFP': '1021',
 'GV0818_Pos12_98_GFP': '1021',
 'GV0818_Pos12_1463_GFP': '1021',
 'GV0815_Pos11_536_GFP': '505',
 'GV0815_Pos11_255_GFP': '532',
 

## 2) Load corresponding cell tracking data and specifiy other parameters

`dataio.hdf5_file_finder(parent_folder_hdf5)` loads a list of all hdf5 files when supplied with the parent directory of all the experiments. This relies on a typical sub-parent directory structure of `'GV****/Pos*/*aligned/HDF/segmented.hdf5'` (aligned folder optional)

In [3]:
### load tracking data
parent_folder_hdf5 = '/home/nathan/data/kraken/scr/h2b/giulia'
hdf5_file_list = dataio.hdf5_file_finder(parent_folder_hdf5)


In [4]:


hdf5_file_path, error_log, success_log = [], [], []
 
for apop_ID in tqdm(event_dict):

    expt = "GV" + str(re.findall(r"GV(\d+)", apop_ID)[0])
    position = re.findall(r"Pos(\d+)", apop_ID)[0]

    position = "Pos" + position

    expt_position = os.path.join(
        expt, position, ""
    )  ## additional '' here so that / added to end of string

    cell_ID = int((re.findall(r"(\d+)_.FP", apop_ID))[0])
    print("ID", apop_ID, "experiment:", expt_position)

    # checks if hdf5 file path already loaded to avoid repeat loading
    if expt_position not in hdf5_file_path:
        ## load that track data
        print("Loading", expt_position)
        hdf5_file_path = [
            hdf5_file_path
            for hdf5_file_path in hdf5_file_list
            if expt_position in hdf5_file_path
        ][0]
        wt_cells, scr_cells, all_cells = dataio.load_tracking_data(hdf5_file_path)
        print("Loaded", expt_position)

    if "RFP" in apop_ID:
        cell_ID = -cell_ID

    focal_time = int(event_dict[apop_ID])

    target_cell = [cell for cell in all_cells if cell.ID == cell_ID][0]

    focal_cell = pd.DataFrame(columns = ['x', 'y', 't', 'ID', 'state'])

    focal_cell['x'], focal_cell['y'],focal_cell['t'], focal_cell['ID'], focal_cell['state'] =target_cell['x'], target_cell['y'],target_cell['t'], target_cell['ID'], target_cell['state']  

    fn = f'/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/apoptoses/apoptotic_tracks/{apop_ID}.csv'

    focal_cell.to_csv(fn)

  0%|          | 0/385 [00:00<?, ?it/s]

[INFO][2022/03/09 12:34:18 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0815/Pos23/Pos23_aligned/HDF/segmented.hdf5...
[INFO][2022/03/09 12:34:18 PM] Loading tracks/obj_type_1


ID GV0815_Pos23_185_GFP experiment: GV0815/Pos23/
Loading GV0815/Pos23/
Btrack version no.: 0.4.3


[INFO][2022/03/09 12:34:20 PM] Loading objects/obj_type_1 (182687, 5) (160127 filtered: area>=100)
[INFO][2022/03/09 12:34:21 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0815/Pos23/Pos23_aligned/HDF/segmented.hdf5
[INFO][2022/03/09 12:34:21 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0815/Pos23/Pos23_aligned/HDF/segmented.hdf5...
[INFO][2022/03/09 12:34:21 PM] Loading tracks/obj_type_2
[INFO][2022/03/09 12:34:22 PM] Loading objects/obj_type_2 (48093, 5) (41816 filtered: area>=100)
[INFO][2022/03/09 12:34:22 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0815/Pos23/Pos23_aligned/HDF/segmented.hdf5
[INFO][2022/03/09 12:34:22 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0816/Pos14/Pos14_aligned/HDF/segmented.hdf5...
[INFO][2022/03/09 12:34:22 PM] Loading tracks/obj_type_1


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0815/Pos23/
ID GV0815_Pos23_634_GFP experiment: GV0815/Pos23/
ID GV0816_Pos14_7_GFP experiment: GV0816/Pos14/
Loading GV0816/Pos14/
Btrack version no.: 0.4.3


...

Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0794/Pos5/
ID GV0797_Pos5_971_GFP experiment: GV0797/Pos5/
Loading GV0797/Pos5/
Btrack version no.: 0.4.3


[INFO][2022/03/09 12:42:31 PM] Loading objects/obj_type_1 (83742, 5) (65837 filtered: area>=100)
[INFO][2022/03/09 12:42:32 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0797/Pos5/Pos5_aligned/HDF/segmented.hdf5
[INFO][2022/03/09 12:42:32 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0797/Pos5/Pos5_aligned/HDF/segmented.hdf5...
[INFO][2022/03/09 12:42:32 PM] Loading tracks/obj_type_2
[INFO][2022/03/09 12:42:32 PM] Loading objects/obj_type_2 (22517, 5) (15314 filtered: area>=100)
[INFO][2022/03/09 12:42:32 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0797/Pos5/Pos5_aligned/HDF/segmented.hdf5
[INFO][2022/03/09 12:42:32 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0817/Pos11/Pos11_aligned/HDF/segmented.hdf5...
[INFO][2022/03/09 12:42:32 PM] Loading tracks/obj_type_1


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0797/Pos5/
ID GV0797_Pos5_741_GFP experiment: GV0797/Pos5/
ID GV0797_Pos5_742_GFP experiment: GV0797/Pos5/
ID GV0817_Pos11_141_GFP experiment: GV0817/Pos11/
Loading GV0817/Pos11/
Btrack version no.: 0.4.3


[INFO][2022/03/09 12:42:34 PM] Loading objects/obj_type_1 (278388, 5) (263033 filtered: area>=100)
[INFO][2022/03/09 12:42:37 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0817/Pos11/Pos11_aligned/HDF/segmented.hdf5
[INFO][2022/03/09 12:42:37 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0817/Pos11/Pos11_aligned/HDF/segmented.hdf5...
[INFO][2022/03/09 12:42:37 PM] Loading tracks/obj_type_2
[INFO][2022/03/09 12:42:38 PM] Loading objects/obj_type_2 (103313, 5) (85256 filtered: area>=100)
[INFO][2022/03/09 12:42:39 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0817/Pos11/Pos11_aligned/HDF/segmented.hdf5


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0817/Pos11/
ID GV0817_Pos11_944_GFP experiment: GV0817/Pos11/
ID GV0817_Pos11_393_GFP experiment: GV0817/Pos11/
ID GV0817_Pos11_2358_GFP experiment: GV0817/Pos11/


In [43]:
### finding all scr apoptoses in uninduced datasets?